In [1]:
# For loading the environment keys

from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
os.getenv("LANGFUSE_HOST") # Confirming use of simpplr host

'https://langfuse.simpplr.xyz'

In [3]:
from langfuse import Langfuse
from langchain_core.prompts import ChatPromptTemplate

# Creating the client that will connect with the project via keys and host
langfuse_client = Langfuse(
    secret_key = os.getenv("LANGFUSE_SECRET_KEY"),
    public_key = os.getenv("LANGFUSE_PUBLIC_KEY"),
    host = os.getenv("LANGFUSE_HOST")
)

In [4]:
# Procedure to create a prompt and load it in the project

langfuse_client.create_prompt(
    name = "teacher-assistant",
    type = "text",
    prompt = "As a {{criticlevel}} teaching expert, are you satisfied with the answer of {{answer}}? Tell in short",
    config = {
        "model" : "llama3:8b", "temperature":0.3, "top_p":0.1
    }
)

In [5]:
# Loading the created prompt from the project

langfuse_prompt = langfuse_client.get_prompt("teacher-assistant", version=1)

# Converting it into langchain type
langchain_prompt = ChatPromptTemplate.from_template(langfuse_prompt.get_langchain_prompt())

In [6]:
#from langfuse.langchain import CallbackHandler # for tracing
from langchain_ollama import ChatOllama

#langfuse_callback_handler = CallbackHandler() # Used for tracing purpose

# Picking up the model configurations from the prompt stored in langfuse. This is optional.
model = langfuse_prompt.config["model"]
temperature = str(langfuse_prompt.config["temperature"])
top_p = str(langfuse_prompt.config["top_p"])

# Using the ChatOllama model for output generation
llm = ChatOllama(model = model, temperature=temperature, top_p = top_p)

# Chaining prompt with model
llm_chain = langchain_prompt | llm

# creating user input
user_input = {
    "criticlevel" : "starting",
    "answer" : "The Earth is flat"
}

# Calling the model with user input for generation
response = llm_chain.invoke(input = user_input,
                           #config = {
                           #    "callbacks":[langfuse_callback_handler]} # Uncomment for tracing
                           )

In [7]:
print(response.content)

No! As a starting teaching expert, I can confidently say that the answer "The Earth is flat" is not accurate and lacks scientific evidence. In fact, overwhelming scientific consensus confirms that the Earth is an oblate spheroid (a slightly flattened sphere). The evidence from various fields such as astronomy, geology, physics, and more all support a round Earth.
